In [1]:
## Libraries
%matplotlib inline
import math
import numpy as np
import matplotlib.pyplot as plt
from insight.pv_model import pv_model_class
import pandas as pd
import os

In [2]:
# Constants and variables

Iscn = 8.21 # nominal short circuit current
Vocn = 32.9 # nominal open circuit voltage
Imp = 7.61 # array current at MPP
Vmp = 26.3 # array voltage at MPP
Pmax_e = Vmp * Imp # experimental array max power output
Kv = -0.123 # voltage temperature coefficient
Ki = 0.0032 # current temperature coefficient
Ns = 54 # number of series cells
Gn = 1000. # nominal irradiance
G = Gn
Tn = 25. + 273.15 # nominal operating temperature
T = Tn

Egap = 1.8 * 10 ** -19 # Bandgap of silicon (silicio cristalino)

In [3]:
df = pd.read_csv(os.path.join("..//..//..//data", "TMY_data_242127_47.69_-121.10_2015.csv"))


/Users/margotadam/opt/anaconda3/envs/real_modeling/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,25,26,27,28,29,30,31,32,33,34,35,36,37,39,40,41,42,43,45) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
new_header = df.iloc[1]
tmy = df[2:]
tmy.columns = new_header
tmy = tmy.dropna(axis=1)
tmy['Hour'][30]

'14'

In [5]:
tmy_july = tmy[(tmy['Month'] == '7') & (tmy['Day'] == '3')]
tmy_jan = tmy[(tmy['Month'] == '1') & (tmy['Day'] == '2')]


In [6]:
### Equations from Masters textbook (table 4.1 and table 4.2)
DNI = int(tmy['DNI'][12795])
DHI = int(tmy['DHI'][12795])
GHI = int(tmy['GHI'][12795])

# ground reflectance
rho = 0.2
# number of days
#month = int(tmy['Month'][12795])
#day = int(tmy['Day'][12795])
#n = (30*month - 30) + day

# calculated by hand for 9/24
n = 267
# hour
hour = int(tmy['Hour'][12795])
# latitude
lat = 47.69
# collector azimuth
az_c = 0
# collector tilt angle
sigma = lat * 0.76 + 3.1
# declination
decl = 23.45 * np.sin(360/365 * (n - 81))
# hour angle
H = 15 * (12 - hour)
# solar altitude
beta = np.arcsin(np.cos(lat) * np.cos(decl) * np.cos(H) + np.sin(lat) * np.sin(decl))
# solar azimuth
az_s = np.arcsin(np.cos(decl) * np.sin(H) / np.cos(beta))

cos_theta = np.cos(beta) * np.cos(az_s - az_c) * np.sin(sigma) + np.sin(beta) * np.cos(sigma)
B = DNI * cos_theta
D = DHI * (1 + np.cos(sigma)) / 2
R = GHI * rho * (1 - np.cos(sigma)) / 2
G = B + D + R
print(G)

250.4324849782308


In [7]:
## Calculating maximum power point for the PV model for consecutive hours throughout a day

pv_mod = pv_model_class.PvModel(Iscn, Vocn, Imp, Vmp, Kv, Ki, Ns, Gn, G, Tn, T, Egap)
# find resistor values and diode ideality constant
Rs, Rp, a = pv_mod.find_resistors()
# find voltage, current, and power arrays
v, i, P = pv_mod.calculate_power(Rs, Rp, a)
# set array size to be 10x10
pv_mod.array_dim = [10,10]
max_power = []

# calculate day number
#    ex. 7/3 -> 184
n = 2 # January 2nd
# ground reflectance (fixed constant)
rho = 0.2
# latitude (for Seattle)
lat = 47.69
# collector azimuth,
# can either get from the dataset or assume 0
az_c = 0
# collector tilt angle (fixed equation)
sigma = lat * 0.76 + 3.1

for i in range(2, 24*2 + 2, 2):
    # i is index of the dataset for the hour we want
    
    # shift to get to month of July 3rd
    # i += 8786
    
    # shift to get to Jan 2nd
    i += 50
    hour = int(tmy['Hour'][i])
    
    DNI = int(tmy['DNI'][i])
    DHI = int(tmy['DHI'][i])
    GHI = int(tmy['GHI'][i])
    pv_mod.G = pv_mod.find_irradiance(DNI, DHI, GHI, rho, n, hour, lat, az_c, sigma)
    
    v, i, P = pv_mod.calculate_power(Rs, Rp, a)
    max_idx = np.argmax(P)
    if P[max_idx] < 0.001:
        P[max_idx] = 0
    max_power.append(P[max_idx])

# each index of max_power represents one hour of the day
print(max_power)

[22228.575, 22228.575, 22228.575, 22228.575, 22228.575, 22228.575, 22228.575, 22228.575, 22462.56, 2099.2417959860422, 1752.6604156572228, 7584.566074977145, 2727.722742408497, 1845.3237469526841, 2936.1913435525116, 207.41012488037265, 22228.575, 22228.575, 22228.575, 22228.575, 22228.575, 22228.575, 22228.575, 22228.575]
